In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image
import cv2


In [3]:
# Set parameters
IMG_HEIGHT = 181
IMG_WIDTH = 181
BATCH_SIZE = 32
NUM_CLASSES = 2  # Black and White pieces
EPOCHS = 10
LEARNING_RATE = 0.0001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'Using device: {DEVICE}')


Using device: cuda


In [5]:
# Load pretrained MobileNetV2
base_model = models.mobilenet_v2(pretrained=True)
base_model.classifier[1] = nn.Linear(base_model.classifier[1].in_features, NUM_CLASSES)
base_model = base_model.to(DEVICE)

# Print model architecture
print(base_model)


c:\Users\yassi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yassi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
%pip install torchsummary
%pip install torchviz
%pip install tensorboardX

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from torchsummary import summary

# Assuming your model is already defined as base_model
summary(base_model, input_size=(3, IMG_HEIGHT, IMG_WIDTH))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 91, 91]             864
       BatchNorm2d-2           [-1, 32, 91, 91]              64
             ReLU6-3           [-1, 32, 91, 91]               0
            Conv2d-4           [-1, 32, 91, 91]             288
       BatchNorm2d-5           [-1, 32, 91, 91]              64
             ReLU6-6           [-1, 32, 91, 91]               0
            Conv2d-7           [-1, 16, 91, 91]             512
       BatchNorm2d-8           [-1, 16, 91, 91]              32
  InvertedResidual-9           [-1, 16, 91, 91]               0
           Conv2d-10           [-1, 96, 91, 91]           1,536
      BatchNorm2d-11           [-1, 96, 91, 91]             192
            ReLU6-12           [-1, 96, 91, 91]               0
           Conv2d-13           [-1, 96, 46, 46]             864
      BatchNorm2d-14           [-1, 96,

In [8]:
from torchviz import make_dot

# Create a dummy input tensor with the same shape as your input data
dummy_input = torch.randn(1, 3, IMG_HEIGHT, IMG_WIDTH).to(DEVICE)

# Generate the graph
out = base_model(dummy_input)
dot = make_dot(out, params=dict(base_model.named_parameters()))
dot.render("mobilenetv2_model", format="png")


'mobilenetv2_model.png'

In [9]:
from tensorboardX import SummaryWriter

writer = SummaryWriter('runs/mobilenet_v2')
dummy_input = torch.randn(1, 3, IMG_HEIGHT, IMG_WIDTH).to(DEVICE)
writer.add_graph(base_model, dummy_input)
writer.close()
